In [1]:
import os
import mlflow
from mlflow import MlflowClient
import torch
import random
import numpy as np
from tqdm import tqdm
import sys

PROJECT_PATH = '/home/gergogaliger/PROJECTS/road_safety_analysis/frequency_analysis/model_development'
SHARED_PROJECT_PATH = '/opt/jupyterhub/SHARED_PROJECTS/road_safety_analysis'

sys.path.append(PROJECT_PATH)

from src.utils.data_utils import read_mlflow_dataset
from src.trainers.AETrainer import AEFCNTrainer

RANDOM_SEED = 42
AE_RUN_ID = '381ad6c4b562468593dda02afa29c719'

mlflow.set_tracking_uri('http://clijupyter01.kozlek.local:4242')

In [2]:
experiment = mlflow.set_experiment('01-02_fully_connected_network.ipynb')

In [3]:
with mlflow.start_run(log_system_metrics=True) as run:
    # Seed random generators to ensure deterministic experiments
    random.seed(RANDOM_SEED)
    np.random.seed(RANDOM_SEED)
    torch.manual_seed(RANDOM_SEED)
    torch.cuda.manual_seed(RANDOM_SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    # Define PyTorch device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Read and log train, validation and test datasets
    data_date = '18-06-2024'
    X_train, y_train, _ = read_mlflow_dataset(SHARED_PROJECT_PATH, data_date, 'train', targets='acc_no_23-24', device=device)
    X_valid, y_valid, _ = read_mlflow_dataset(SHARED_PROJECT_PATH, data_date, 'validate', targets='acc_no_23-24', device=device)
    X_test, y_test, non_accident_dim = read_mlflow_dataset(SHARED_PROJECT_PATH, data_date, 'test', targets='acc_no_23-24', device=device)

    # Load trained autoencoder from MLflow
    autoencoder = mlflow.pytorch.load_model(f'runs:/{AE_RUN_ID}/{MlflowClient().list_artifacts(AE_RUN_ID, "models")[0].path}')
    
    # Specify and log training parameters
    params = {
        'autoencoder': autoencoder,
        'inp_dim': X_train.shape[1] - non_accident_dim + autoencoder.enc_dim,
        'learning_rate': 1e-2
    }
    mlflow.log_params(params)

    # Define, train and evaluate model
    trainer = AEFCNTrainer(**params)
    trainer.train(X_train, y_train, X_valid, y_valid, 30)
    trainer.evaluate(X_test, y_test)

2024/06/24 18:41:15 WARNING mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics because creating `GPUMonitor` failed with error: `pynvml` is not installed, to log GPU metrics please run `pip install pynvml` to install it..
2024/06/24 18:41:15 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.
/opt/conda/envs/python3/lib/python3.11/site-packages/mlflow/data/dataset_source_registry.py:150: UserWarning: The specified dataset source can be interpreted in multiple ways: LocalArtifactDatasetSource, LocalArtifactDatasetSource. MLflow will assume that this is a LocalArtifactDatasetSource source.
  return _dataset_source_registry.resolve(
/opt/conda/envs/python3/lib/python3.11/site-packages/mlflow/data/digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  string_columns = trimmed_df.columns[(df.applymap(type) == str).all(0)]
/opt/conda/envs/python3/lib/python3.11/site-packages/mlflow/data/dat

2024/06/24 18:41:17 INFO mlflow.store.artifact.artifact_repo: The progress bar can be disabled by setting the environment variable MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR to false
  0%|          | 0/30 [00:00<?, ?it/s]/home/gergogaliger/.local/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([28395])) that is different to the input size (torch.Size([28395, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/gergogaliger/.local/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([3549])) that is different to the input size (torch.Size([3549, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
100%|██████████| 30/30 [00:33<00:00,  1.13s/it]
/home/g